# Process single cell profiles

In [1]:
import pathlib
import pprint

import pandas as pd

from pycytominer import annotate, normalize, feature_select

In [2]:
# Path to dir with nuclei feature files
converted_dir = pathlib.Path("./data/converted_profiles")

# output path for single cell profiles 
output_dir = pathlib.Path("./data/single_cell_profiles")
output_dir.mkdir(parents=True, exist_ok=True)  

# Extract the plate names from the file name
plate_names = [file.stem.replace("_converted", "") for file in converted_dir.glob("*.parquet")]

# path for platemap directory
platemap_dir = pathlib.Path("../metadata/")

# operations to perform for feature selection
feature_select_ops = [
    "variance_threshold",
    "correlation_threshold",
    "blocklist",
]

In [3]:
# create plate info dictionary 
plate_info_dictionary = {
    name: {
        "profile_path": str(
            pathlib.Path(list(converted_dir.rglob(f"{name}_converted.parquet"))[0]).resolve(
                strict=True
            )
        ),
        "platemap_path": str(
            pathlib.Path(list(platemap_dir.rglob(f"{name}_platemap.csv"))[0]).resolve(
                strict=True
            )
        ),
    }
    for name in plate_names
}

# view the dictionary to assess that all info is added correctly
pprint.pprint(plate_info_dictionary, indent=4)

{   'localhost220512140003_KK22-05-198': {   'platemap_path': '/home/jenna/CFReT_data/metadata/localhost220512140003_KK22-05-198_platemap.csv',
                                             'profile_path': '/home/jenna/CFReT_data/3.process_cfret_features/data/converted_profiles/localhost220512140003_KK22-05-198_converted.parquet'},
    'localhost220513100001_KK22-05-198_FactinAdjusted': {   'platemap_path': '/home/jenna/CFReT_data/metadata/localhost220513100001_KK22-05-198_FactinAdjusted_platemap.csv',
                                                            'profile_path': '/home/jenna/CFReT_data/3.process_cfret_features/data/converted_profiles/localhost220513100001_KK22-05-198_FactinAdjusted_converted.parquet'},
    'localhost230405150001': {   'platemap_path': '/home/jenna/CFReT_data/metadata/localhost230405150001_platemap.csv',
                                 'profile_path': '/home/jenna/CFReT_data/3.process_cfret_features/data/converted_profiles/localhost230405150001_converted.

In [4]:
for plate, info in plate_info_dictionary.items():
    print(f"Performing pycytominer pipeline for {plate}")
    output_annotated_file = str(pathlib.Path(f"{output_dir}/{plate}_sc_annotated.parquet"))
    output_normalized_file = str(pathlib.Path(f"{output_dir}/{plate}_sc_normalized.parquet"))
    output_feature_select_file = str(pathlib.Path(f"{output_dir}/{plate}_sc_feature_selected.parquet"))

    profile_df = pd.read_parquet(info["profile_path"])
    platemap_df = pd.read_csv(info["platemap_path"])

    # Step 1: Annotation
    annotate(
        profiles=profile_df,
        platemap=platemap_df,
        join_on=["Metadata_well_position", "Image_Metadata_Well"],
        output_file=output_annotated_file,
        output_type="parquet",
    )

    # Load the annotated parquet file to fix metadata columns names
    annotated_df = pd.read_parquet(output_annotated_file)

    # Rename columns using the rename() function
    column_name_mapping = {
        "Image_Metadata_Site": "Metadata_Site",
    }

    annotated_df.rename(columns=column_name_mapping, inplace=True)

    # Save the modified DataFrame back to the same location
    annotated_df.to_parquet(output_annotated_file, index=False)

    # Step 2: Normalization
    normalized_df = normalize(
        profiles=output_annotated_file,
        method="standardize",
        output_file=output_normalized_file,
        output_type="parquet",
    )

    # Step 3: Feature selection
    feature_select(
        output_normalized_file,
        operation=feature_select_ops,
        output_file=output_feature_select_file,
        output_type="parquet"
    )
    print(f"Annotation, normalization, and feature selection have been performed for {plate}")

Performing pycytominer pipeline for localhost230405150001
Annotation, normalization, and feature selection have been performed for localhost230405150001
Performing pycytominer pipeline for localhost220512140003_KK22-05-198
Annotation, normalization, and feature selection have been performed for localhost220512140003_KK22-05-198
Performing pycytominer pipeline for localhost220513100001_KK22-05-198_FactinAdjusted
Annotation, normalization, and feature selection have been performed for localhost220513100001_KK22-05-198_FactinAdjusted


In [5]:
# Check output file
test_df = pd.read_parquet(output_feature_select_file)

print(test_df.shape)
test_df.head(2)

(17995, 664)


,Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_treatment,Metadata_dose,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,Metadata_Image_Count_Cells,Metadata_ImageNumber,Metadata_Plate,...,Nuclei_Texture_InverseDifferenceMoment_PM_3_01_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_03_256,Nuclei_Texture_SumEntropy_ER_3_03_256,Nuclei_Texture_SumEntropy_Hoechst_3_03_256,Nuclei_Texture_SumEntropy_Mitochondria_3_03_256,Nuclei_Texture_SumEntropy_PM_3_01_256,Nuclei_Texture_SumVariance_ER_3_03_256,Nuclei_Texture_SumVariance_Hoechst_3_01_256,Nuclei_Texture_SumVariance_Mitochondria_3_03_256,Nuclei_Texture_SumVariance_PM_3_03_256
0,A,9,9,drug_x,5uM,423.043530,103.325259,34,1.0,localhost220513100001_KK22-05-198_FactinAdjusted,...,0.140046,0.177754,-1.035615,-0.499102,-0.927906,-0.378530,-0.452070,-0.309250,-0.179711,-0.196718
1,A,9,9,drug_x,5uM,163.177967,171.310538,34,1.0,localhost220513100001_KK22-05-198_FactinAdjusted,...,-1.392301,-1.901882,1.253318,0.375069,1.911982,1.972299,1.347963,-0.199487,0.628766,0.613891
